In [1]:
import csv
from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt

In [2]:
##############################################################################
# parameters #################################################################
##############################################################################

# A, paths
data_path = "D:\\University\\DataScienceWorkshop\\"    #change to your path
train_path = data_path + 'clicks_train.csv'            # path to training file
test_path = data_path + 'clicks_test.csv'              # path to testing file
submission_path = data_path + 'sub_proba.csv'          # path of to be outputted submission file

# B, model
alpha = .1  # learning rate
beta = 0.   # smoothing parameter for adaptive learning rate
L1 = 0.    # L1 regularization, larger value means more regularized
L2 = 0.     # L2 regularization, larger value means more regularized

# C, feature/hash trick
D = 2 ** 20             # number of weights to use
interaction = False     # whether to enable poly2 feature interactions

# D, training/validation
epoch = 1       # learn training data for N passes
holdafter = None   # data after date N (exclusive) are used as validation
holdout = None  # use every N training instance for holdout validation

In [3]:
##############################################################################
# class, function, generator definitions #####################################
##############################################################################

class ftrl_proximal(object):
    ''' Our main algorithm: Follow the regularized leader - proximal

        In short,
        this is an adaptive-learning-rate sparse logistic-regression with
        efficient L1-L2-regularization

        Reference:
        http://www.eecs.tufts.edu/~dsculley/papers/ad-click-prediction.pdf
    '''

    def __init__(self, alpha, beta, L1, L2, D, interaction):
        # parameters
        self.alpha = alpha
        self.beta = beta
        self.L1 = L1
        self.L2 = L2

        # feature related parameters
        self.D = D
        self.interaction = interaction

        # model
        # n: squared sum of past gradients
        # z: weights
        # w: lazy weights
        self.n = [0.] * D
        self.z = [0.] * D
        self.w = {}

    def _indices(self, x):
        ''' A helper generator that yields the indices in x

            The purpose of this generator is to make the following
            code a bit cleaner when doing feature interaction.
        '''

        # first yield index of the bias term
        yield 0

        # then yield the normal indices
        for index in x:
            yield index

        # now yield interactions (if applicable)
        if self.interaction:
            D = self.D
            L = len(x)

            x = sorted(x)
            for i in xrange(L):
                for j in xrange(i+1, L):
                    # one-hot encode interactions with hash trick
                    yield abs(hash(str(x[i]) + '_' + str(x[j]))) % D

    def predict(self, x):
        ''' Get probability estimation on x

            INPUT:
                x: features

            OUTPUT:
                probability of p(y = 1 | x; w)
        '''

        # parameters
        alpha = self.alpha
        beta = self.beta
        L1 = self.L1
        L2 = self.L2

        # model
        n = self.n
        z = self.z
        w = {}

        # wTx is the inner product of w and x
        wTx = 0.
        for i in self._indices(x):
            sign = -1. if z[i] < 0 else 1.  # get sign of z[i]

            # build w on the fly using z and n, hence the name - lazy weights
            # we are doing this at prediction instead of update time is because
            # this allows us for not storing the complete w
            if sign * z[i] <= L1:
                # w[i] vanishes due to L1 regularization
                w[i] = 0.
            else:
                # apply prediction time L1, L2 regularization to z and get w
                w[i] = (sign * L1 - z[i]) / ((beta + sqrt(n[i])) / alpha + L2)

            wTx += w[i]

        # cache the current w for update stage
        self.w = w

        # bounded sigmoid function, this is the probability estimation
        return 1. / (1. + exp(-max(min(wTx, 35.), -35.)))

    def update(self, x, p, y):
        ''' Update model using x, p, y

            INPUT:
                x: feature, a list of indices
                p: click probability prediction of our model
                y: answer

            MODIFIES:
                self.n: increase by squared gradient
                self.z: weights
        '''

        # parameter
        alpha = self.alpha

        # model
        n = self.n
        z = self.z
        w = self.w

        # gradient under logloss
        g = p - y

        # update z and n
        for i in self._indices(x):
            sigma = (sqrt(n[i] + g * g) - sqrt(n[i])) / alpha
            z[i] += g - sigma * w[i]
            n[i] += g * g


def logloss(p, y):
    ''' FUNCTION: Bounded logloss

        INPUT:
            p: our prediction
            y: real answer

        OUTPUT:
            logarithmic loss of p given y
    '''

    p = max(min(p, 1. - 10e-15), 10e-15)
    return -log(p) if y == 1. else -log(1. - p)


def data(path, D):
    ''' GENERATOR: Apply hash-trick to the original csv row
                   and for simplicity, we one-hot-encode everything

        INPUT:
            path: path to training or testing file
            D: the max index that we can hash to

        YIELDS:
            ID: id of the instance, mainly useless
            x: a list of hashed and one-hot-encoded 'indices'
               we only need the index since all values are either 0 or 1
            y: y = 1 if we have a click, else we have y = 0
    '''

    for t, row in enumerate(DictReader(open(path))):
        # process id
        disp_id = int(row['display_id'])
        ad_id = int(row['ad_id'])

        # process clicks
        y = 0.
        if 'clicked' in row:
            if row['clicked'] == '1':
                y = 1.
            del row['clicked']

        x = []
        for key in row:
            x.append(abs(hash(key + '_' + row[key])) % D)

        row = prcont_dict.get(ad_id, [])		
        # build x
        ad_doc_id = -1
        for ind, val in enumerate(row):
            if ind==0:
                ad_doc_id = int(val)
            x.append(abs(hash(prcont_header[ind] + '_' + val)) % D)

        row = event_dict.get(disp_id, [])
        ## build x
        disp_doc_id = -1
        for ind, val in enumerate(row):
            if ind==0:
                uuid_val = val
            if ind==1:
                disp_doc_id = int(val)
            x.append(abs(hash(event_header[ind] + '_' + val)) % D)

        if (ad_doc_id in leak_uuid_dict) and (uuid_val in leak_uuid_dict[ad_doc_id]):
            x.append(abs(hash('leakage_row_found_1'))%D)
        else:
            x.append(abs(hash('leakage_row_not_found'))%D)
            
        yield t, disp_id, ad_id, x, y

In [4]:
##############################################################################
# start training #############################################################
##############################################################################

start = datetime.now()

# initialize ourselves a learner
learner = ftrl_proximal(alpha, beta, L1, L2, D, interaction)

print("Content..")
with open(data_path + "promoted_content.csv") as infile:
	prcont = csv.reader(infile)
	#prcont_header = (prcont.next())[1:]
	prcont_header = next(prcont)[1:]
	prcont_dict = {}
	for ind,row in enumerate(prcont):
		prcont_dict[int(row[0])] = row[1:]
		if ind%100000 == 0:
			print(ind)
#		if ind==10000:
#			break
	print(len(prcont_dict))
del prcont

print("Events..")
with open(data_path + "events.csv") as infile:
	events = csv.reader(infile)
	#events.next()
	next(events)
	event_header = ['uuid', 'document_id', 'platform', 'geo_location', 'loc_country', 'loc_state', 'loc_dma']
	event_dict = {}
	for ind,row in enumerate(events):
		tlist = row[1:3] + row[4:6]
		loc = row[5].split('>')
		if len(loc) == 3:
			tlist.extend(loc[:])
		elif len(loc) == 2:
			tlist.extend( loc[:]+[''])
		elif len(loc) == 1:
			tlist.extend( loc[:]+['',''])
		else:
			tlist.append(['','',''])	
		event_dict[int(row[0])] = tlist[:] 
		if ind%100000 == 0:
			print("Events : ", ind)
#		if ind==10000:
#			break
	print(len(event_dict))
del events

print("Leakage file..")
leak_uuid_dict= {}

Content..
0
100000
200000
300000
400000
500000
559583
Events..
Events :  0
Events :  100000
Events :  200000
Events :  300000
Events :  400000
Events :  500000
Events :  600000
Events :  700000
Events :  800000
Events :  900000
Events :  1000000
Events :  1100000
Events :  1200000
Events :  1300000
Events :  1400000
Events :  1500000
Events :  1600000
Events :  1700000
Events :  1800000
Events :  1900000
Events :  2000000
Events :  2100000
Events :  2200000
Events :  2300000
Events :  2400000
Events :  2500000
Events :  2600000
Events :  2700000
Events :  2800000
Events :  2900000
Events :  3000000
Events :  3100000
Events :  3200000
Events :  3300000
Events :  3400000
Events :  3500000
Events :  3600000
Events :  3700000
Events :  3800000
Events :  3900000
Events :  4000000
Events :  4100000
Events :  4200000
Events :  4300000
Events :  4400000
Events :  4500000
Events :  4600000
Events :  4700000
Events :  4800000
Events :  4900000
Events :  5000000
Events :  5100000
Events :  520000

In [7]:
prcont_dict

{1: ['6614', '1', '7'],
 2: ['471467', '2', '7'],
 3: ['7692', '3', '7'],
 4: ['471471', '2', '7'],
 5: ['471472', '2', '7'],
 6: ['12736', '1', '7'],
 7: ['12808', '1', '7'],
 8: ['471477', '2', '7'],
 9: ['13379', '1', '7'],
 10: ['13885', '1', '7'],
 11: ['14230', '1', '7'],
 12: ['446701', '10', '19'],
 13: ['471499', '10', '19'],
 14: ['471500', '10', '19'],
 15: ['471501', '10', '19'],
 16: ['471514', '17', '19'],
 17: ['471517', '10', '19'],
 18: ['471518', '10', '19'],
 19: ['471519', '5', '19'],
 20: ['446660', '21', '19'],
 21: ['20896', '16', '19'],
 22: ['21203', '10', '19'],
 23: ['21112', '10', '19'],
 24: ['20100', '10', '19'],
 25: ['25921', '1', '7'],
 26: ['471529', '17', '19'],
 27: ['471530', '17', '19'],
 28: ['471534', '10', '19'],
 29: ['471536', '12', '19'],
 30: ['471539', '10', '19'],
 31: ['471542', '10', '19'],
 32: ['23989', '23', '19'],
 33: ['26447', '24', '19'],
 34: ['29075', '24', '19'],
 35: ['29658', '24', '19'],
 36: ['29658', '24', '19'],
 37: ['20

In [ ]:
# start training
for e in range(epoch):
    loss = 0.
    count = 0
    date = 0

    for t, disp_id, ad_id, x, y in data(train_path, D):  # data is a generator
        #    t: just a instance counter
        # date: you know what this is
        #   ID: id provided in original data
        #    x: features
        #    y: label (click)

        # step 1, get prediction from learner
        p = learner.predict(x)

        if (holdafter and date > holdafter) or (holdout and t % holdout == 0):
            # step 2-1, calculate validation loss
            #           we do not train with the validation data so that our
            #           validation loss is an accurate estimation
            #
            # holdafter: train instances from day 1 to day N
            #            validate with instances from day N + 1 and after
            #
            # holdout: validate with every N instance, train with others
            loss += logloss(p, y)
            count += 1
        else:
            # step 2-2, update learner with label (click) information
            learner.update(x, p, y)

        if t%1000000 == 0:
            print("Processed : ", t, datetime.now())
#        if t == 100000: 
#            break

Processed :  0 2017-01-18 11:28:31.945176
Processed :  1000000 2017-01-18 11:30:05.287581
Processed :  2000000 2017-01-18 11:31:31.586349
Processed :  3000000 2017-01-18 11:32:57.964170
Processed :  4000000 2017-01-18 11:34:26.010696
Processed :  5000000 2017-01-18 11:35:54.075335
Processed :  6000000 2017-01-18 11:37:20.186957
Processed :  7000000 2017-01-18 11:38:45.727935
Processed :  8000000 2017-01-18 11:40:11.431802
Processed :  9000000 2017-01-18 11:41:36.520708
Processed :  10000000 2017-01-18 11:43:05.908411


In [ ]:
##############################################################################
# start testing, and build Kaggle's submission file ##########################
##############################################################################

with open(submission, 'w') as outfile:
    outfile.write('display_id,ad_id,clicked\n')
    for t, disp_id, ad_id, x, y in data(test_path, D):
        p = learner.predict(x)
        outfile.write('%s,%s,%s\n' % (disp_id, ad_id, str(p)))
        if t%1000000 == 0:
            print("Processed : ", t, datetime.now())
#        if t ==100000:
#            break